In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [4]:

if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')
  proj_dir = "/content/drive/MyDrive/ece884_project/"
else:
  proj_dir = "../"

df = pd.read_csv(f"{proj_dir}data_clean/clean.csv")
column_names = df.columns
df = df.to_numpy()

Mounted at /content/drive


In [5]:
import os
import re
gans_saved = os.listdir(f"{proj_dir}saved_models/")
model_number = [int(re.sub("gan", "", x)) for x in gans_saved]
last_model = max(model_number)

In [6]:
columns_in_generated = 98
rows_in_generated = 500

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, mean_squared_error

In [8]:
def logistic_reg_confution(X, y):
    return LogisticRegression().fit(X, y).predict(X)

In [9]:
def logistic_reg_single_col(X, y):
  n_columns = X.shape[1]
  scores = []
  for column in range(n_columns):
    x_sub = X[:, column].reshape((len(y), 1))
    yhat = LogisticRegression().fit(x_sub, y).predict(x_sub)
    mse = mean_squared_error(y, yhat)
    scores.append(np.sum(mse))
  return scores

In [19]:
results = []
for i in range(1, last_model+1):
  gan_path = f"{proj_dir}saved_models/gan{i}"
  # print(gan_path)

  last_gan = tf.saved_model.load(gan_path)
  noise = tf.random.normal(shape=[rows_in_generated, columns_in_generated])
  generated_data = last_gan(noise)
  
  generated_label = np.concatenate([np.ones((rows_in_generated, )),
                                    np.zeros((rows_in_generated, ))], axis=0)
  
  random_index = np.random.permutation(df.shape[0])
  real_data = df[random_index, :] # sample from full dataset

  generated_and_real = np.concatenate([generated_data, real_data], axis=0)

  random_index = np.random.permutation(2*rows_in_generated)
  generated_and_real_shuffeled = generated_and_real[random_index, :]
  generated_label_shuffeled = generated_label[random_index]

  rows = logistic_reg_single_col(generated_and_real_shuffeled, generated_label_shuffeled)
  results.append(rows)
#  conf = confusion_matrix(generated_label_shuffeled, yhat)
# print(conf)

In [20]:
pd.DataFrame(results, columns=column_names)

,hospnum,rdelay,sex,age,rsleep,ratrial,rct,rvisinf,rhep24,rasp3,...,dead8,h14,isc14,nk14,strk14,hti14,pe14,dvt14,tran14,ncb14
0,0.000,0.005,0.217,0.0,0.355,0.386,0.203,0.362,0.490,0.373,...,0.495,0.495,0.484,0.489,0.468,0.500,0.497,0.499,0.496,0.487
1,0.008,0.001,0.227,0.0,0.357,0.406,0.164,0.340,0.493,0.399,...,0.494,0.494,0.493,0.498,0.485,0.496,0.498,0.499,0.498,0.492
2,0.000,0.000,0.249,0.0,0.338,0.406,0.170,0.335,0.014,0.382,...,0.491,0.495,0.489,0.497,0.481,0.499,0.499,0.499,0.497,0.490
3,0.000,0.005,0.245,0.0,0.345,0.403,0.160,0.350,0.484,0.389,...,0.497,0.495,0.489,0.494,0.478,0.497,0.497,0.500,0.496,0.486
4,0.000,0.001,0.232,0.0,0.365,0.413,0.341,0.329,0.486,0.367,...,0.496,0.496,0.481,0.496,0.474,0.500,0.495,0.000,0.495,0.487
5,0.006,0.000,0.241,0.0,0.357,0.066,0.160,0.337,0.482,0.096,...,0.008,0.250,0.489,0.495,0.480,0.497,0.008,0.500,0.499,0.491
6,0.014,0.000,0.283,0.0,0.361,0.086,0.156,0.334,0.493,0.395,...,0.011,0.497,0.491,0.492,0.480,0.497,0.498,0.500,0.495,0.482
7,0.008,0.003,0.218,0.0,0.367,0.395,0.161,0.334,0.491,0.375,...,0.492,0.007,0.014,0.008,0.472,0.003,0.498,0.500,0.499,0.488
8,0.012,0.001,0.224,0.0,0.354,0.395,0.357,0.328,0.486,0.387,...,0.496,0.490,0.009,0.497,0.478,0.498,0.495,0.500,0.497,0.485
9,0.007,0.003,0.247,0.0,0.362,0.401,0.157,0.352,0.483,0.388,...,0.489,0.492,0.490,0.491,0.473,0.496,0.497,0.499,0.496,0.487
